In [1]:
pip install findspark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=1a1110c51ea8bbace8fb6c5ae37b4965ba557973dc8fa34f7d48fddf3c053bfd
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
# Import findspark and initialize.
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSessionl
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
# from pyspark import SparkFiles
# url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

import os
import urllib
from pyspark.sql import SparkSession

# URL of the CSV file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Local file path where the data will be downloaded
local_directory = "/path/to/your/local/folder"
local_file_path = os.path.join(local_directory, "home_sales_revised.csv")

# Create the local directory if it does not exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# Download the file using urllib
urllib.request.urlretrieve(url, local_file_path)

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Read the data from the local file into a DataFrame
df = spark.read.csv("file://" + local_file_path, header=True, inferSchema=True)

# Show the DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePrice").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for a four-bedroom house sold in each year
query = """
SELECT YEAR(date) as year, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()



+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePriceByYearBuilt").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for homes with 3 bedrooms and 3 bathrooms by year built
query = """
SELECT date_built as year_built, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [9]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePriceByYearBuilt").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for homes with 3 bedrooms, 3 bathrooms, two floors,
# and are greater than or equal to 2,000 square feet, by year built
query = """
SELECT date_built as year_built, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# start_time = time.time()



# print("--- %s seconds ---" % (time.time() - start_time))


# Assuming you already have the DataFrame 'df' created from the previous code

# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Measure the start time of the query execution
start_time = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000
query = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df = spark.sql(query)

# Extract the "average_price" value from the DataFrame result
view_rating = float(result_df.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time = time.time()

# Show the "view" rating for the average price (rounded to two decimal places)
print("View Rating for Average Price (rounded to two decimal places): {:.2f}".format(view_rating))

# Calculate and display the runtime of the query
runtime = end_time - start_time
print("Query Runtime:", runtime, "seconds")




View Rating for Average Price (rounded to two decimal places): 473796.26
Query Runtime: 0.5700302124023438 seconds


In [11]:
# 7. Cache the the temporary table home_sales.

# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Measure the start time of the query execution
start_time = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000
query = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df = spark.sql(query)

# Extract the "average_price" value from the DataFrame result
view_rating = float(result_df.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time = time.time()

# Show the "view" rating for the average price (rounded to two decimal places)
print("View Rating for Average Price (rounded to two decimal places): {:.2f}".format(view_rating))

# Calculate and display the runtime of the query
runtime = end_time - start_time
print("Query Runtime:", runtime, "seconds")


View Rating for Average Price (rounded to two decimal places): 473796.26
Query Runtime: 2.0433547496795654 seconds


In [12]:
# 8. Check if the table is cached.
# spark.catalog.isCached('home_sales')

# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales_view")

# Print the result
print("Is 'home_sales_view' cached?", is_cached)


Is 'home_sales_view' cached? True


In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# start_time = time.time()



# print("--- %s seconds ---" % (time.time() - start_time))


# Assuming you already have the DataFrame 'df' created from the previous code

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Measure the start time of the query execution
start_time_cached = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000 using the cached data
query_cached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_cached = spark.sql(query_cached)

# Extract the "average_price" value from the DataFrame result
view_rating_cached = float(result_df_cached.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time_cached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) using cached data
print("View Rating for Average Price (rounded to two decimal places) with cached data: {:.2f}".format(view_rating_cached))

# Calculate and display the runtime of the query using cached data
runtime_cached = end_time_cached - start_time_cached
print("Query Runtime with cached data:", runtime_cached, "seconds")

# Measure the start time of the query execution without using cached data
start_time_uncached = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000 without using cached data
query_uncached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_uncached = spark.sql(query_uncached)

# Extract the "average_price" value from the DataFrame result
view_rating_uncached = float(result_df_uncached.collect()[0]["average_price"])

# Measure the end time of the query execution without using cached data
end_time_uncached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) without using cached data
print("View Rating for Average Price (rounded to two decimal places) without cached data: {:.2f}".format(view_rating_uncached))

# Calculate and display the runtime of the query without using cached data
runtime_uncached = end_time_uncached - start_time_uncached
print("Query Runtime without cached data:", runtime_uncached, "seconds")



View Rating for Average Price (rounded to two decimal places) with cached data: 473796.26
Query Runtime with cached data: 0.2787609100341797 seconds
View Rating for Average Price (rounded to two decimal places) without cached data: 473796.26
Query Runtime without cached data: 0.22475290298461914 seconds


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Assuming you have the DataFrame 'df' created and formatted as Parquet from the previous code

# Write the DataFrame to Parquet format and partition by the "date_built" field
df.write.partitionBy("date_built").parquet("/path/to/parquet_output")


In [15]:
# 11. Read the formatted parquet data.
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("ReadParquetData").getOrCreate()

# Read the Parquet data with partitions
parquet_data = spark.read.format("parquet").load("/path/to/parquet_output")

# Show the data
parquet_data.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [16]:
# 12. Create a temporary table for the parquet data

parquet_data.createOrReplaceTempView("parquet_data_view")


In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# start_time = time.time()



# print("--- %s seconds ---" % (time.time() - start_time))





# Assuming you have the DataFrame 'parquet_data' created from the previous code

# Register the DataFrame as a temporary view
parquet_data.createOrReplaceTempView("parquet_data_view")

# Measure the start time of the query execution
start_time_uncached = time.time()

# SQL query to calculate the average price for homes greater than or equal to $350,000
query_uncached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM parquet_data_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_uncached = spark.sql(query_uncached)

# Extract the "average_price" value from the DataFrame result
view_rating_uncached = float(result_df_uncached.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time_uncached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) without using cached data
print("View Rating for Average Price (rounded to two decimal places) without cached data: {:.2f}".format(view_rating_uncached))

# Calculate and display the runtime of the query without using cached data
runtime_uncached = end_time_uncached - start_time_uncached
print("Query Runtime without cached data:", runtime_uncached, "seconds")


View Rating for Average Price (rounded to two decimal places) without cached data: 473796.26
Query Runtime without cached data: 0.48806285858154297 seconds


In [18]:
# 14. Uncache the home_sales temporary table.

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Unpersist the cached DataFrame
df.unpersist()

# Check if the table is still cached (should return False)
is_cached = df.is_cached
print("Is 'home_sales' cached?", is_cached)




Is 'home_sales' cached? False


In [19]:
# 15. Check if the home_sales is no longer cached

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Unpersist the cached DataFrame
df.unpersist()

# Check if the DataFrame is still cached
is_cached = df.is_cached

if not is_cached:
    print("DataFrame 'home_sales' is no longer cached.")
else:
    print("DataFrame 'home_sales' is still cached.")




DataFrame 'home_sales' is no longer cached.
